# Covid Vaccine Coverage Prediction - an Update
This project is to use the themes of each county in United States to predict the outcome of vaccine coverage.
The target is to predict the vaccine rate, which is updated daily. We used the data up to 7/21/2021 for calibration
The purpose if to identify the area of concerns


In [54]:
from pandas import Series, DataFrame
import pandas as pd
%pylab inline
import IPython
import pandas as pd
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import sklearn as sk
import statsmodels.formula.api as smf
import statsmodels.api as sm
import os
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from matplotlib.lines import Line2D
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import time
from pandas import Series, DataFrame
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib
Populating the interactive namespace from numpy and matplotlib


In [55]:
#Daily vaccination administration data from cdc up to 7/31
#source:https://data.cdc.gov/Vaccinations/COVID-19-Vaccinations-in-the-United-States-Jurisdi/unsk-b7fc
vac_admin = pd.read_csv('vac_county.csv')

#Vaccination hesitance survey result by states and county
#source: https://data.cdc.gov/stories/s/cnd2-a6zw
vac_h = pd.read_csv('vac_hesi_county.csv')
vac_h=vac_h.dropna()

#Vaccination coverage index by states and county
#source:https://vaccine.precisionforcovid.org/
vac_c = pd.read_csv('cvac_county.csv')
vac_c_sub=pd.read_csv('cvac_sub_county.csv')

#2020 Us census data for state population
#source: https://www.census.gov/data/tables/2020/dec/2020-apportionment-data.html
# pop = pd.read_csv('pop_county.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0,1,3,4,22,23,24,25,26) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [56]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

def state_abbrev(x):
    
    '''
    This function converts full US tates name into abbreviated version.
    '''
    for key, value in us_state_abbrev.items():
        if x==key:
            return value

def state_full(x):
    '''
    This function converts abbreviated US tates name into full version.
    '''
    for key, value in us_state_abbrev.items():
        if x==value:
            return key


In [57]:
vac_admin
vac_admin['state_county'] = vac_admin['Recip_State'].str.lower() + '_' + vac_admin['Recip_County'].str.lower()
vac_admin.head()

,Date,FIPS,MMWR_week,Recip_County,Recip_State,Series_Complete_Pop_Pct,Series_Complete_Yes,Series_Complete_12Plus,Series_Complete_12PlusPop_Pct,Series_Complete_18Plus,...,Administered_Dose1_Recip_18Plus,Administered_Dose1_Recip_18PlusPop_Pct,Administered_Dose1_Recip_65Plus,Administered_Dose1_Recip_65PlusPop_Pct,SVI_CTGY,Series_Complete_Pop_Pct_SVI,Series_Complete_12PlusPop_Pct_SVI,Series_Complete_18PlusPop_Pct_SVI,Series_Complete_65PlusPop_Pct_SVI,state_county
0,7/31/2021,18119,30.0,Owen County,IN,34.9,7255.0,7255.0,40.1,7043.0,...,7403.0,44.9,2881.0,69.5,Low-Mod,Low-Mod VC/Low-Mod SVI,Mod-High VC/Low-Mod SVI,Mod-High VC/Low-Mod SVI,Mod-High VC/Low-Mod SVI,in_owen county
1,7/31/2021,21147,30.0,McCreary County,KY,30.9,5326.0,5326.0,36.0,5253.0,...,5836.0,43.1,1810.0,64.6,High,Low-Mod VC/High SVI,Low-Mod VC/High SVI,Low-Mod VC/High SVI,Mod-High VC/High SVI,ky_mccreary county
2,7/31/2021,27103,30.0,Nicollet County,MN,50.8,17399.0,17354.0,59.1,16304.0,...,17333.0,64.8,4865.0,85.0,Low-Mod,High VC/Low-Mod SVI,High VC/Low-Mod SVI,High VC/Low-Mod SVI,High VC/Low-Mod SVI,mn_nicollet county
3,7/31/2021,19183,30.0,Washington County,IA,46.9,10308.0,10307.0,56.0,9892.0,...,10442.0,62.9,3767.0,85.9,Low,Mod-High VC/Low SVI,High VC/Low SVI,High VC/Low SVI,High VC/Low SVI,ia_washington county
4,7/31/2021,8023,30.0,Costilla County,CO,34.0,1321.0,1318.0,38.5,1245.0,...,NaN,0.0,NaN,0.0,High,Low-Mod VC/High SVI,Low-Mod VC/High SVI,Low-Mod VC/High SVI,Low VC/High SVI,co_costilla county


In [58]:
vac_h['state_county'] = vac_h['state'].str.lower() + '_' + vac_h['county'].str.lower() + ' county'
vac_h.head()

,county,state,"% Estimated Hesitant or Unsure - May 26 - June 7, 2021","% Estimated Hesitant- May 26 - June 7, 2021","% Estimated Strongly Hesitant- May 26 - June 7, 2021",state_county
0,Autauga,AL,22%,17%,13%,al_autauga county
1,Baldwin,AL,20%,15%,12%,al_baldwin county
2,Barbour,AL,23%,17%,13%,al_barbour county
3,Bibb,AL,25%,18%,14%,al_bibb county
4,Blount,AL,24%,19%,14%,al_blount county


In [59]:
vac_c['state2'] = vac_c['state'].map(state_abbrev)
vac_c['state_county'] = vac_c['state2'].str.lower() + '_' + vac_c['county'].str.lower()
vac_c.head()

,FIPS,state,county,Th1_Historic_Undervaccination,Th2_Sociodemographic_Barriers,Th3_Resource_constrained_Healthcare_System,Th4_Healthcare_Accessibility_Barriers,Th5_Irregular_Care_Seeking_Behavior,CVAC,state2,state_county
0,1001,Alabama,Autauga County,0.419612,0.231455,0.827762,0.484241,0.830309,0.607768,AL,al_autauga county
1,1003,Alabama,Baldwin County,0.228590,0.150271,0.170965,0.391277,0.726839,0.227953,AL,al_baldwin county
2,1005,Alabama,Barbour County,0.505571,0.991404,0.621777,0.966253,0.694683,0.885068,AL,al_barbour county
3,1007,Alabama,Bibb County,0.410060,0.888252,0.335880,0.842089,0.834448,0.764406,AL,al_bibb county
4,1009,Alabama,Blount County,0.440306,0.716014,0.850684,0.580388,0.837950,0.795288,AL,al_blount county


In [60]:
merg2=vac_admin.merge(vac_h,
              left_on='state_county',
              right_on='state_county')
merg2=merg2.merge(vac_c,
               left_on='state_county',
               right_on='state_county')
merg2

,Date,FIPS_x,MMWR_week,Recip_County,Recip_State,Series_Complete_Pop_Pct,Series_Complete_Yes,Series_Complete_12Plus,Series_Complete_12PlusPop_Pct,Series_Complete_18Plus,...,FIPS_y,state_y,county_y,Th1_Historic_Undervaccination,Th2_Sociodemographic_Barriers,Th3_Resource_constrained_Healthcare_System,Th4_Healthcare_Accessibility_Barriers,Th5_Irregular_Care_Seeking_Behavior,CVAC,state2
0,7/31/2021,18119,30.0,Owen County,IN,34.9,7255.0,7255.0,40.1,7043.0,...,18119,Indiana,Owen County,0.937281,0.568927,0.773639,0.240369,0.436485,0.659981,IN
1,7/31/2021,21147,30.0,McCreary County,KY,30.9,5326.0,5326.0,36.0,5253.0,...,21147,Kentucky,McCreary County,0.205030,0.904489,0.185291,0.595989,0.248647,0.388411,KY
2,7/31/2021,27103,30.0,Nicollet County,MN,50.8,17399.0,17354.0,59.1,16304.0,...,27103,Minnesota,Nicollet County,0.274435,0.104744,0.690226,0.357529,0.690863,0.380452,MN
3,7/31/2021,19183,30.0,Washington County,IA,46.9,10308.0,10307.0,56.0,9892.0,...,19183,Iowa,Washington County,0.026106,0.229545,0.161414,0.165871,0.014008,0.012098,IA
4,7/31/2021,8023,30.0,Costilla County,CO,34.0,1321.0,1318.0,38.5,1245.0,...,8023,Colorado,Costilla County,0.339382,0.979306,0.700096,0.692773,0.578478,0.760586,CO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2990,7/31/2021,40147,30.0,Washington County,OK,30.9,15906.0,15906.0,36.6,15386.0,...,40147,Oklahoma,Washington County,0.475963,0.305317,0.368036,0.774594,0.424387,0.458771,OK
2991,7/31/2021,39159,30.0,Union County,OH,48.8,28794.0,28793.0,58.1,27043.0,...,39159,Ohio,Union County,0.089780,0.006686,0.652658,0.074499,0.138809,0.051894,OH
2992,7/31/2021,40029,30.0,Coal County,OK,29.3,1608.0,1608.0,34.5,1592.0,...,40029,Oklahoma,Coal County,0.539000,0.834448,0.707736,0.951289,0.599172,0.846864,OK
2993,7/31/2021,47077,30.0,Henderson County,TN,29.7,8354.0,8354.0,34.8,8239.0,...,47077,Tennessee,Henderson County,0.836995,0.769500,0.760904,0.710920,0.815027,0.908946,TN


In [64]:
vac_c[vac_c['county']== 'Washington County']

,FIPS,state,county,Th1_Historic_Undervaccination,Th2_Sociodemographic_Barriers,Th3_Resource_constrained_Healthcare_System,Th4_Healthcare_Accessibility_Barriers,Th5_Irregular_Care_Seeking_Behavior,CVAC,state2,state_county
64,1129,Alabama,Washington County,0.496339,0.943012,0.717287,0.556192,0.822350,0.822350,AL,al_washington county
182,5143,Arkansas,Washington County,0.281121,0.409105,0.128303,0.630372,0.863419,0.445718,AR,ar_washington county
305,8121,Colorado,Washington County,0.405603,0.238459,0.758994,0.225406,0.411016,0.349570,CO,co_washington county
386,12133,Florida,Washington County,0.996180,0.775868,0.893028,0.753900,0.712512,0.952245,FL,fl_washington county
536,13303,Georgia,Washington County,0.420567,0.922318,0.695320,0.941102,0.622095,0.836995,GA,ga_washington county
594,16087,Idaho,Washington County,0.389366,0.812480,0.273480,0.836995,0.667622,0.667940,ID,id_washington county
689,17189,Illinois,Washington County,0.167463,0.176695,0.299586,0.116842,0.560013,0.127030,IL,il_washington county
784,18175,Indiana,Washington County,0.923273,0.629099,0.583254,0.636422,0.485196,0.750716,IN,in_washington county
880,19183,Iowa,Washington County,0.026106,0.229545,0.161414,0.165871,0.014008,0.012098,IA,ia_washington county
988,20201,Kansas,Washington County,0.893346,0.567972,0.628781,0.125756,0.181789,0.473098,KS,ks_washington county


# Population

In [ ]:
pop.columns=['state','county','population']

In [ ]:
pop['st']=pop['state'].map(state_abbrev)
pop['st_cty']=pop['st']+' '+pop['county']

In [ ]:
pop=pop[['st_cty','population']]
pop

In [ ]:
print('pop shape {} \n'.format(pop.shape))

county_list=pop['county'].tolist()
pop['st_cty']=pop['state']+

len(county_list)

# Vaccination status

In [ ]:
vac_admin.columns

In [ ]:
vac_admin['Administered_Dose1_Pop_Pct'].sort_values(ascending=False)

In [ ]:
temp=vac_admin.groupby(['Recip_State','Recip_County'])['Administered_Dose1_Pop_Pct'].agg('max')

temp=temp.reset_index()
temp['st_cty']=temp['Recip_State']+' '+temp['Recip_County']
vac=temp[['st_cty','Administered_Dose1_Pop_Pct']]
vac.columns=['st_cty','vac_rate']
vac

# Vaccine hesitation

In [ ]:
cty_name=list(vac['st_cty'])

def cty_match(x,y):
    for name in cty_name:
        if x in name:
            if y in name:
                return(name)
                        
#to test:
cty_match('Harris','TX')

In [ ]:
vac['st_cty']

In [ ]:
vac_h.T

In [ ]:

def f(x):
    return cty_match(x['county'],x['state'])

vac_h.T.apply(f)

In [ ]:
vac_h['st_cty']=vac_h[['state','county']].apply(cty_match)

In [ ]:
vac_h[['state','county']].apply

In [ ]:
vac_c['st']=vac_c['state'].map(state_abbrev)
vac_c['st_cty']=vac_c['st']+' '+vac_c['county']
temp1=vac_c[['st_cty',
             'Th1_Historic_Undervaccination',
             'Th2_Sociodemographic_Barriers',
             'Th3_Resource_constrained_Healthcare_System',
             'Th4_Healthcare_Accessibility_Barriers',
             'Th5_Irregular_Care_Seeking_Behavior']]
temp1

In [ ]:
vac_c_sub['st']=vac_c['state'].map(state_abbrev)
vac_c_sub['st_cty']=vac_c['st']+' '+vac_c['county']
vac_c_sub.columns
temp2=vac_c_sub[['st_cty',
                 'Th1_Lower Coverage and Higher Refusal Rates',
       'Th2_Lack of Access to Infomation',
       'Th2_Socio-economically disadvantaged',
       'Th3_Low Healthcare System Capacity', 'Th3_Weak Healthcare System',
       'Th4_Barriers due to cost', 'Th4_Barriers due to Transportation',
       'Th5_Lack of Designated Medical Home',
       'Th5_Lack of Routine Care Visits',
                ]]
temp2

In [ ]:
vac_2model=temp.merge(temp1,
                    on='st_cty')
vac_2model

In [ ]:
vac_2model.sort_values(by='vac_rate',ascending=False)

# Linear regression

In [ ]:
# Special packages
import statsmodels.api as sm
from patsy import dmatrices


y, X = dmatrices('vac_rate ~ Th1_Historic_Undervaccination + Th2_Sociodemographic_Barriers + Th3_Resource_constrained_Healthcare_System +Th4_Healthcare_Accessibility_Barriers+Th5_Irregular_Care_Seeking_Behavior', data=vac_2model, return_type='dataframe')


model = sm.OLS(y, X)       # Set up the model
result = model.fit()       # Fit model (find the intercept and slopes)
print(result.summary())

In [ ]:
# Special packages
import statsmodels.api as sm
from patsy import dmatrices


y, X = dmatrices('Administered_Dose1_Pop_Pct ~ Th1_Historic_Undervaccination + Th2_Sociodemographic_Barriers + Th3_Resource_constrained_Healthcare_System +Th5_Irregular_Care_Seeking_Behavior', data=vac_2model, return_type='dataframe')


model = sm.OLS(y, X)       # Set up the model
result = model.fit()       # Fit model (find the intercept and slopes)
print(result.summary())

In [ ]:
# Special packages
import statsmodels.api as sm
from patsy import dmatrices


y, X = dmatrices('Administered_Dose1_Pop_Pct ~ CVAC', data=vac_2model, return_type='dataframe')


model = sm.OLS(y, X)       # Set up the model
result = model.fit()       # Fit model (find the intercept and slopes)
print(result.summary())

# Random Forest